# Supervised Learning with Feedforward Neural Networks

Supervised machine learning is the type you probably already know about. The basic idea is that you have a set of inputs and a set of outputs, and you want to learn the relationship between them. For example, you might have a set of images of handwritten digits and you want to learn to recognize which digit is in each image. Or you might have a set of emails and you want to learn to recognize which ones are spam. Or you might have a set of audio clips and you want to learn to recognize what words were spoken in each clip.

*TLDR:* in supervised machine learning, we train an algorithm or neural network to predict outputs, given inputs.

The two basic types of supervised learning are:
- *Classfication:* outputs are discrete categories. For example, types of particles in the Standard Model, or handwritten digits.
- *Regression:* outputs are continuous or allow for interpolation, such as the mass of a particle or the price of a house.

## Classification: Some Simple Math Problems

We'll exemplify the idea in a simple problem: determining whether the product of two numbers $q_1$ and $q_2$ is positive.

We'll start by generating the data that models this problem.

Now we will set up the neural network. This takes in 2 numbers and will output one. 

You'll notice alternative sequences of linear layers and tanh non-linearities, similar to earlier in the course, but now we end with sigmoid:
$$\sigma(x) = \frac{1}{1+e^{-x}}$$
which is a non-linear function that maps any real number to a number between 0 and 1. This is useful for classification problems, since we can interpret the output as a probability.

Now set up some hyperparameters related to learning, the optimizer, and the loss function.

### **Loss function**
 we choose is typical for a binary classification problem such as this one. It is called the *binary cross entropy* and is defined as
$$L = -\frac{1}{N}\sum_{i=1}^N y_i \log(\hat{y}_i) + (1-y_i)\log(1-\hat{y}_i)$$
where $y_i$ is the true label (0 or 1) and $\hat{y}_i$ is the predicted label (between 0 and 1).

The basic idea of this loss is that if the true label is 1, then we want to maximize $\hat{y}_i$, and if the true label is 0, then we want to maximize $1-\hat{y}_i$. This is a good loss function for classification problems because it penalizes the network more for being very wrong than for being a little bit wrong. The value inside the log is effectively never exactly 0 or 1, so we don't have to worry about multiplying by infinity.

### **Optimizer**
We choose to use stochastic gradient descent, which differs from gradient descent due to mini-batches. The idea is that instead of computing the gradient of the loss function over the entire dataset, we compute it over a small subset of the dataset called a mini-batch. This is much faster, and the gradient is still a good approximation to the true gradient. The word "stochastic" refers to the fact that the mini-batches are often chosen randomly (though there are other ways for stochasticity to enter).

Time for the train loop.

### Three Classes

In this example we have three classes instead of two. Instead of sigmoid in the last layer, we will use softmax, which is a generalization of sigmoid to multiple classes. It is defined as
$$\sigma_i(x) = \frac{e^{x_i}}{\sum_{j=1}^N e^{x_j}}$$
where $x_i$ is the $i^\text{th}$ element of the input vector $x$ and $N$ is the number of classes. The output of the softmax is a vector of length $N$ whose elements sum to 1. We can interpret each element as the probability that the input belongs to the corresponding class.

Instead of binary cross-entropy loss, we'll use a natural generalization called *categorical cross-entropy* loss, which is defined as

$$L = -\frac{1}{N}\sum_{i=1}^N y_{i,j} \sum_j \log(\hat{y}_{i,j})$$

where $y_{i,j}$ is the true label (0 or 1) for the $i^{th}$ input and $j^{th}$ class, and $\hat{y}_{i,j}$ is the predicted label (between 0 and 1) for the $i^{th}$ input and $j^{th}$ class.

Why does this reproduce the binary cross entropy in the $j=2$ case? (Remember: y's are interpreted as probabilities).

## Regression: $Z \to e^+ e^-$ decays

In this problem we'll use CERN OpenData taken from the website [opendata.cern.ch](http://opendata.cern.ch/record/545) to train a classifier to distinguish between $Z \to e^+ e^-$ decays and background events. The data is stored in the file `Zee.csv` and contains the following variables:
- `E1`, `E2`: The energy of the two electrons in GeV
- `pt1`, `pt2`: The transverse momentum of the two electrons in GeV
- `eta1`, `eta2`: The pseudorapidity of the two electrons
- `phi1`, `phi2`: The azimuthal angle of the two electrons
and some other data that won't be as relevant for us.

We will train a neural network to predict the invariant mass of the Z-boson from the transverse momenta `pt`, the pseudo-rapidities `eta` and the azimuthal angles `phi` of the electron and positron. The pseudo-rapidity is given by
$$\eta = -\ln(\tan(\theta/2))$$
where $\theta$ is the angle between the particle's momentum and the beam axis.
Of course, the exact formula for the invariant mass is known, but this is useful pedagogically because it will give us some confidence in the neural network.

In terms of these variables, the formula for the reconstructed invariant mass of the $Z$-boson is 
$M_Z^2 = 2 p_{T,1} p_{T,2} ( \cosh(\eta_1 - \eta_2) - \cos(\phi_1 - \phi_2))$.


We begin with some imports and implementing a function for the invariant mass.

### Attempt 1: No normalization on the inputs

### Attempt 2: Normalize the data a bit

### Attempt 3: Use the Physically Sensible Variables